In [1]:
import pandas as pd
import browser_cookie3
import nest_asyncio
import ujson as json
nest_asyncio.apply()
from datetime import datetime

from fetch import better_search_bofa_global_research, bofa_download_reports

%load_ext autoreload
%autoreload 2

In [14]:
cj = browser_cookie3.chrome()

In [15]:
cookie_2165_string = r"your 2165 cookie string here" 
session_cookie_string = r"your session cookies here" 
encrypted_session_query_string = r"your encryted session token here" 

In [16]:
df1 = better_search_bofa_global_research(
    search_string="rates",
    datetime_from=datetime(2024, 2, 21),
    datetime_to=datetime(2024, 2, 23),
    cookie_2165=cookie_2165_string,
    encrypted_session_query=encrypted_session_query_string,
    xlsx_path="bofa_research_search_results.xlsx",
    run_analyst_lookup=True,
)
df1.head()

Search Result Matches:  21


,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
0,"Angelakis, Ioannis|Martin, Barnaby|Dennis, Wil...",|MERDOTWSOD,MERDOTWSOD,2024-02-23T12:26:57.000Z,6208-12663253-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,9,415776,Follow The Flow: The reach for beta strengthens,...,2024-02-25T20:28:33.245Z,,,,|EUROCR|1B934106|,Inflows continue uninterrupted for another wee...,,||,||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1,"Global Rates, Research|Satko, Erjon",|MERDOTWSOD,MERDOTWSOD,2024-02-23T11:04:47.000Z,6208-12663224-1,MLCY|EXCN|MLPY|EXPY|MLMY|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,6,327288,Global Rates Watch: EGB spreads: opportunities...,...,2024-02-25T20:28:33.246Z,,,,|1B934122|1B934123|AUST1S|RAGB1S|BGB1S|BGBRT1S...,Tightening trend in EGBs does not stand out re...,,|||||||||||||||||||||||||||||||,|||||||||||||||||||||||||||||||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
2,"Hoexter, Ken|Roszkowski, Adam|103480",|MERDOTWSOD,MERDOTWSOD,2024-02-23T10:01:13.000Z,6208-12663055-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,12,970151,Transportation - Trucking: Survey Says: Demand...,...,2024-02-25T20:28:33.246Z,,,,|1B934115|ANALYTIC|JBHT|KNX|ODFL|SAIA|XPO|,Truck Shipper Survey #303: BofA Demand Indicat...,,|||27596|2000027350|64821|6547|12008,|||172296|178775|220649|249882|264622,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3,"Feniger, Michael|El-Sabbahy, Sherif|Nayar, Nan...",|MERDOTWSOD,MERDOTWSOD,2024-02-23T10:00:16.000Z,6208-12663113-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,7,610469,NV5 Global Inc.: Solid finish to the year – su...,...,2024-02-25T20:28:33.246Z,NVEE,|233630939,|67966773,NVEE|1B934115|,NV5 reported a solid Q4 (EBITDA $37mn vs cons ...,,|233630939|,|67966773|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
4,"European Rates, Research|Salim, Sphia|Braizinh...",|MERDOTWSOD,MERDOTWSOD,2024-02-23T06:30:18.000Z,6208-12663061-1,MLCY|EXCN|MLPY|EXPY|MLMY|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,5,188121,European Rates Alpha: Target reached on 3m1y E...,...,2024-02-25T20:28:33.246Z,,,,|EURATRI|1B934122|1B934123|,We close the 3m1y payer spread we had entered ...,,|||,|||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...


In [33]:
df2 = better_search_bofa_global_research(
    datetime_from=datetime(2024, 1, 1),
    datetime_to=datetime(2024, 2, 25),
    cookie_2165=cookie_2165_string,
    encrypted_session_query=encrypted_session_query_string,
    run_analyst_lookup=False,
    xlsx_path="bofa_research_search_results_today.xlsx",
    verbose=True
)
df2.head()

Will Fetch 3 payloads
[{'queries': [{'rowCount': 2500, 'firstRow': 0, 'arguments': [{'field': 'MLExternal', 'conditions': [{'operator': 'EqualTo', 'values': ['TRUE']}]}, {'field': 'DocumentDate', 'conditions': [{'operator': 'GreaterThanEqualTo', 'values': ['2024-01-01 00:00:00']}, {'operator': 'LessThanEqualTo', 'values': ['2024-01-22 00:00:00']}]}, {'field': 'FreeText', 'conditions': [{'operator': 'EqualTo', 'values': ['']}]}]}]}, {'queries': [{'rowCount': 2500, 'firstRow': 0, 'arguments': [{'field': 'MLExternal', 'conditions': [{'operator': 'EqualTo', 'values': ['TRUE']}]}, {'field': 'DocumentDate', 'conditions': [{'operator': 'GreaterThanEqualTo', 'values': ['2024-01-22 00:00:00']}, {'operator': 'LessThanEqualTo', 'values': ['2024-02-12 00:00:00']}]}, {'field': 'FreeText', 'conditions': [{'operator': 'EqualTo', 'values': ['']}]}]}]}, {'queries': [{'rowCount': 2500, 'firstRow': 0, 'arguments': [{'field': 'MLExternal', 'conditions': [{'operator': 'EqualTo', 'values': ['TRUE']}]}, {'fi

,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
0,102041|102716|103523|103293|,|MERDOTWSOD,MERDOTWSOD,2024-01-22T23:33:41.000Z,6208-12649185-1,MLCN|EXCN|MLPY|EXPY|MLMN|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,9,917193,Situation Room: New frontier in flatness,...,2024-02-25T20:35:06.819Z,,,,|SITROOM|1B93460|1B934106|,In January IG non-financial spread curves have...,,|||,|||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1,102684|102607|102947|103546|,|MERDOTWSOD,MERDOTWSOD,2024-01-22T23:17:57.000Z,6208-12649187-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,3,181166,Homebuilders and Building Products: State of U...,...,2024-02-25T20:35:06.819Z,,,,|1B934115|,We hosted webcast with Zonda to discuss state ...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
2,103105|103450|103264|103448|102960|,|MERDOTWSOD,MERDOTWSOD,2024-01-22T22:34:05.000Z,6208-12649179-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,5,290079,Jack Henry & Associates: Expect CEO transition...,...,2024-02-25T20:35:06.819Z,JKHY,|56837,|173769,JKHY|1B934115|,"Earlier, JKHY announced current COO & Presiden...",,|56837|,|173769|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3,103035|102725|103311|103343|103503|,|MERDOTWSOD,MERDOTWSOD,2024-01-22T22:33:54.000Z,6208-12649040-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,5,289911,Evotec SE: Former CEO’s departure linked to be...,...,2024-02-25T20:35:06.819Z,EVO,|55167,|243702,EVO|1B934115|,"On a conference call earlier today, EVO provid...",,|55167|,|243702|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
4,103350|103349|103383|103110|,|MERDOTWSOD,MERDOTWSOD,2024-01-22T22:03:39.000Z,6208-12649165-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,9,466927,Software: AI Evolution: Cost Savings > Revenue...,...,2024-02-25T20:35:06.819Z,,,,|1B934115|,Expert call addresses potential AI-driven oper...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...


In [18]:
substrings = ["ml", "olui2", "fs"]
ml_cookies = { cookie.name: cookie.value for cookie in cj if any(substring in cookie.domain for substring in substrings) }
print(json.dumps(ml_cookies, indent=4, sort_keys=True))

{
    "COOKIE_SUPPORT": "true",
    "CookieConsent": "{stamp:%27-1%27%2Cnecessary:true%2Cpreferences:true%2Cstatistics:true%2Cmarketing:true%2Cmethod:%27implied%27%2Cver:1%2Cutc:1708613786061%2Cregion:%27US%27}",
    "GUEST_LANGUAGE_ID": "en_US",
    "GZIP": "1",
    "OptanonConsent": "isGpcEnabled=0&datestamp=Fri+Feb+23+2024+12%3A15%3A19+GMT-0600+(Central+Standard+Time)&version=202308.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=c8529af5-0b0a-411c-94ad-f46a5f7cffc9&interactionCount=1&landingPath=NotLandingPage&groups=CCBA%3A1%2CC0001%3A1%2CC0002%3A1%2CC0003%3A1&AwaitingReconsent=false",
    "Wildcat": "NuOqj7fM+St\/faFjAlnzmZ4UMX8uoZUi7WpKAKiylPQzxjDjiJpg93lQhqELIN8TfF+3vx4B1bW93sIPQXYlTP5jyS\/uZT1rKPQ0dtMEt3g=",
    "__hstc": "92415682.e50794566e4bf440f2971b0572895009.1708269805043.1708269805043.1708269805043.1",
    "__pdst": "cbfb41fbbdff4ffa98c3b94bc1ba3cf2",
    "_biz_flagsA": "%7B%22Version%22%3A1%2C%22ViewThrough%22%3A%221%22%2C%22XDomain%22%3A%221%22%7D",
    "_biz_

In [34]:
session_cookie_list = session_cookie_string.split("; ")
session_cookies_dict = {}
for item in session_cookie_list:
    key, value = item.split("=", 1)
    session_cookies_dict[key] = value
    
ml_cookies.update(session_cookies_dict)

df2["documentDate"] = pd.to_datetime(df2["documentDate"])
grouped = df2.groupby(df2["documentDate"].dt.date)
df2s = {group: group_df for group, group_df in grouped}


In [35]:
# TODO parallelize this
for key, df2 in df2s.items():
    print(key)
    display(df2)
    bofa_download_reports(df2, cookies=ml_cookies, verbose=True)

2024-01-01


,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
1282,303502|303688|302696|302836|102106|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 21:00:22+00:00,6208-12639726-1,MLCY|EXCN|MLPY|EXPY|MLMY|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,18,770319,Asia FI & FX Strategy Viewpoint: A deep dive i...,...,2024-02-25T20:35:06.860Z,,,,|EMSPOT|PORT0018|1B240131|1B934109|1B934112|PO...,The rising deficit has eroded Hong Kong's fisc...,,||||||,||||||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1283,302437|303384|303658|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 17:34:05+00:00,6208-12640006-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,8,424980,China Property: Dec. contracted sales early re...,...,2024-02-25T20:35:06.860Z,,,,|1B934115|,"According to CRIC, top 100 developers' contrac...",,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1284,302861|303210|303549|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 17:30:03+00:00,6208-12640004-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,6,376883,Greater China EV and EV battery: China “Future...,...,2024-02-25T20:35:06.860Z,,,,|1B934115|,"On January 1, XPeng officially launched X9, it...",,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1285,302861|303210|303549|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 17:26:40+00:00,6208-12640011-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,5,282258,"Greater China Auto, EV and EV battery: China E...",...,2024-02-25T20:35:06.860Z,,,,|1B934115|,BYD reported Dec sales at 341k units (new high...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1286,303561|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 17:20:22+00:00,6208-12640017-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPY|,MERDOTWSOD,8,460720,Autos - India: Dec: More misses than hits. 2Ws...,...,2024-02-25T20:35:06.860Z,,,,|1B934115|,Dec soft on low seasonality & inventory destoc...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1287,303583|303688|302836|102106|,|MERDOTWSOD,MERDOTWSOD,2024-01-01 16:30:50+00:00,6208-12640000-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,6,369964,Korea Watch: Dec Trade: Export growth rose in ...,...,2024-02-25T20:35:06.860Z,,,,|PORT0097|1B934108|1B934109|1B934112|1B934107|,Dec export growth rose for the 3rd month at 5....,,|||||,|||||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...


c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\Asia_FI_&_FX_Strategy_Viewpoint_A_deep_dive_into_Hong_Kong’s_finances.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\China_Property_Dec._contracted_sales_early_read;_BJSH_home_sales_improved_post_easing.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\Greater_China_EV_and_EV_battery_China_“Future_Car”_Review_#36_XPeng_X9_–_a_flagship_MPV_aims_high.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\Greater_China_Auto,_EV_and_EV_battery_China_EV_brands’_Dec_sales_most_are_good,_but_Li_Auto_&_Aion_outperformed.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\Autos_-_India_Dec_More_misses_than_hits._2Ws,CVs_decent_prints;_PVs_&_tractors_lag.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-01\Korea_Watch_Dec_

,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
1226,101539|101550|102127|103278|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 23:16:11+00:00,6208-12640304-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,7,433905,"Ansys, Inc.: Press reports Ansys in merger talks",...,2024-02-25T20:35:06.858Z,ANSS,|35746,|45960,ANSS|1B934115|CDNS|SNPS|,"According to press reports, Ansys has received...",,|35746||55289|38878,|45960||72225|256951,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1227,104018|102774|102559|103052|103454|103435|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 21:33:32+00:00,6208-12640292-2,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,7,397765,High Quality & Dividend Yield: Screen for January,...,2024-02-25T20:35:06.858Z,,,,|1B934129|1B934144|ACN|GD|GPC|NOC|PKG|PG|,The High Quality & Dividend Yield Screen (HQ&D...,,|||20504|1115|45708|39420|35964|4724,|||37839|136432|141100|215890|229969|228324,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1228,104018|102774|102559|103052|103454|103435|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 21:01:08+00:00,6208-12640252-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,11,535091,Alpha Surprise: Screen for January,...,2024-02-25T20:35:06.858Z,,,,|1B934129|1B934144|ANALYTIC|,"The ASM is a ""Quantamental"" model, which scree...",,|||,|||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1229,311003|302414|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 21:00:08+00:00,6208-12640225-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,4,228104,Coverage Update: Equity Termination of Coverage,...,2024-02-25T20:35:06.858Z,,,,|1B934115|ARRLF|XDMDF|XWEUF|EVRZF|XURTF|XZARF|...,We are terminating coverage of the stocks show...,,||2000028304|234321286|435838860|2000041074|64...,||40219986|104237794|436897758|39564004|646777...,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1230,103110|103383|103240|103508|103542|103460|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 20:48:19+00:00,6208-12640267-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,13,821203,"US Utilities & IPPs: PowerPoints: PNM M&A, PEG...",...,2024-02-25T20:35:06.858Z,,,,|1B934115|ES|IDA|OGE|PNM|PPL|PEG|,PNM merger terminated by AGR. We see attractiv...,,||36883|41584|11442|2000017535|31353|65156,||218204|152762|220999|231631|232832|227307,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1231,102541|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 20:17:03+00:00,6208-12640274-1,MLCY|EXMY|MLIY|EXIY|DSPN|EXCY|MLPY|EXPY|MLMY|,MERDOTWSOD,4,206638,Termination of Coverage: Equity Termination of...,...,2024-02-25T20:35:06.859Z,,,,|1B934115|SGEN|,,,||36417,||532067948,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1232,102975|103365|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 20:14:43+00:00,6208-12640281-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,8,402719,Industry Report: Tracking returns in Consumer ...,...,2024-02-25T20:35:06.859Z,,,,|1B934124|,Year-end rally sustained into Dec but Consumer...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1233,102503|102513|103489|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 19:42:53+00:00,6208-12640271-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,8,625247,Universal Health Services: Approval in Nevada ...,...,2024-02-25T20:35:06.859Z,UHS,|7764,|276301,UHS|1B934115|HCA|,Nevada Medicaid program changes approved will ...,,|7764||2000031123,|276301||30079931,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
1234,101500|103050|103432|103510|103484|302955|,|MERDOTWSOD,MERDOTWSOD,2024-01-02 19:05:31+00:00,6208-12640260-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,10,683030,Commercial Aerospace: Boeing and Airbus potent...,...,2024-02-25T20:35:06.859Z,,,,|1B934115|BA|ERJ|,"According to AAP, Boeing successfully met its ...",,||16799|8565,||56858|100151,https://olui2.fs.ml.com/MDWS

c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\Healthcare_Technology_&_Distribution_Year_Ahead_2024_Weight_loss_and_reimbursement…gains.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\Medical_Technology_Medtech_in_2024_focus_on_estimates_after_the_big_2023_multiple_reset.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\Nextracker_Inc_Don’t_get_lost_in_the_broader_shuffle_–_Flagging_further_upside.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\Investment_Grade_Healthcare_Quarterly_2024_IG_Healthcare_Year_Ahead.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\US_High_Grade_Credit_Research_Top_US_High_Grade_Ideas_–_January_2024_edition.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-01-02\US_Utilities_&_IPPs_Entering_2024_with_new_utility_leaders_at_th

...

c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\US_Biopharmaceuticals_The_Priority_Review_–_Week_of_February_19th.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\Credit_Market_Strategist_What’s_working.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\Pulp_&_Paper_Price_Tracker_Arauco_held_pulp_prices_unchanged_in_March.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\Brady_Corp._-_Cl_A_Post_F2Q24_Organic_growth_a_touch_lighter,_but_mix_provides_support;_Buy.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\Nutrien_Outlook_has_upside_potential;_Remain_constructive.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\Industrial_Weekly_The_Nuts_and_Bolts.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-23\US_Bioph

,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
3124,302973|102134|102499|103470|103475|306004|3026...,|MERDOTWSOD,MERDOTWSOD,2024-02-24 01:06:26+00:00,6208-12663464-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,28,2657123,Systematic Flows Monitor: CTAs see gains on eq...,...,2024-02-25T20:35:06.213Z,,,,|1B934143|,NEW: SPX option gamma positioning; delta-hedge...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...


Creating c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-24
BofA Global Research Report Downloaded - 2024-02-24 - Systematic_Flows_Monitor_CTAs_see_gains_on_equity_longs;_Introducing_SPX_option_gamma_flows
Fetch Finished
2024-02-25


,analysts,clientCodes,creditClientCode,documentDate,documentKey,entitlements,equityClientCode,filePageCount,fileSizeBytes,headline,...,positionChangeDate,primarySymbol,primaryWSODCompany,primaryWSODIssue,symbols,synopsis,urlOfDoc,wsodCompany,wsodIssue,reportLink
3112,303548|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 20:30:39+00:00,6208-12663539-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,6,332986,KT: Key takeaways from Korea Corporate call se...,...,2024-02-25T20:35:06.213Z,KTCNF,|11823,|13683,KTCNF|KT|1B934115|,Reaffirmed improving shareholder returns over ...,,|11823|11823|,|13683|180149|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3113,302750|302017|303677|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 20:30:35+00:00,6208-12663131-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,13,984675,MINISO Group Holdings: Solid finish to 2023; g...,...,2024-02-25T20:35:06.213Z,MNSO,|620763002,|623526913,MNSO|XKTAF|1B934115|,Expect solid finish to 2023 in Dec qtr: 50%+ Y...,,|620763002|620763002|,|623526913|738748164|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3114,102749|302491|302675|103287|102106|302836|3027...,|MERDOTWSOD,MERDOTWSOD,2024-02-25 19:46:23+00:00,6208-12663530-1,MLCY|EXCN|MLPY|EXPY|MLMY|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,14,1419543,What’s priced in: BOI will likely remain on ho...,...,2024-02-25T20:35:06.213Z,,,,|EMQUANT|1B240131|1B934109|1B934112|PORT0114|,Israel's BOI will likely keep the rate on hold...,,|||||,|||||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3115,302356|303655|303720|303315|303692|303614|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 19:00:09+00:00,6208-12663526-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,8,801311,Woodside Energy Group: 15% sell down in Scarbo...,...,2024-02-25T20:35:06.213Z,WOPEF,|9269,|287154,WOPEF|1B934115|,Woodside has agreed to sell a 15.1% interest i...,,|9269|,|287154|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3116,303219|303363|303538|303661|303584|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 18:43:22+00:00,6208-12663545-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,22,1836009,Basic Materials - China: Coal price gain on ti...,...,2024-02-25T20:35:06.213Z,,,,|1B934115|,Coal: Price up amid temporary supply deficit; ...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3117,303502|302836|102106|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 17:21:59+00:00,6208-12663540-1,MLCY|EXCN|MLPY|EXPY|MLMY|EXMN|MLIY|EXIY|DSPN|,MERDOTWSOD,8,230331,EM Alpha: Closing receive 2y US vs HK IRS,...,2024-02-25T20:35:06.213Z,,,,|EMLDM|1B240131|1B934109|1B934112|PORT0212|,The continuation of strong US data has resulte...,,|||||,|||||,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3118,102960|103105|103304|103512|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 15:00:27+00:00,6208-12663449-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,5,292476,"OneMain Holdings, Inc.: BofA Securities 2024 F...",...,2024-02-25T20:35:06.213Z,OMF,|234276624,|68235827,OMF|1B934115|,"This week we hosted OneMain's CEO, Doug Shulma...",,|234276624|,|68235827|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3119,303184|301915|302511|303550|303390|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 14:59:57+00:00,6208-12663517-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,9,574880,Semiconductors - Asia-Pacific: Together strong...,...,2024-02-25T20:35:06.213Z,,,,|1B934115|IBIDF|SNEJF|TSM|TSMWF|,JASM opening marks an important step to both T...,,||52854|3034|63790|63790,||21453|26572|272831|19203,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3120,302437|303384|303658|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 14:34:44+00:00,6208-12663532-1,MLCY|EXCY|MLPY|EXPY|MLMY|EXMY|MLIY|EXIY|DSPN|,MERDOTWSOD,6,327702,China Property: New home sales volume in major...,...,2024-02-25T20:35:06.213Z,,,,|1B934115|,Primary home sales volume in 33 cities decline...,,|,|,https://olui2.fs.ml.com/MDWSODUtility/PdfLoade...
3121,101575|103434|103427|,|MERDOTWSOD,MERDOTWSOD,2024-02-25 14:25:45+00:00,6208-12663456-1,MLCY|EXCY|MLPY

c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\MINISO_Group_Holdings_Solid_finish_to_2023;_growth_likely_to_normalize_in_2024,_but_still_strong.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\What’s_priced_in_BOI_will_likely_remain_on_hold_this_week;_NBH_to_cut.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\Woodside_Energy_Group_15%_sell_down_in_Scarborough_a_positive,_further_sell_downs_expected.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\Basic_Materials_-_China_Coal_price_gain_on_tight_supply;_more_resumption_after_Lantern_festival.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\EM_Alpha_Closing_receive_2y_US_vs_HK_IRS.pdf already exists - exiting
c:\Users\chris\sell-side-research-public\bofa\reports\2024-02-25\OneMain_Holdings,_Inc._BofA_Securities_2024_Financials_Conference_Take